### 8.03. Making Objects Support the Context-Management Protocol

In [1]:
from socket import socket, AddressFamily, SocketKind, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family:AddressFamily=AF_INET, type:SocketKind=SOCK_STREAM):
        self.address = address
        # self.family = AF_INET
        self.family = family
        # self.type = SOCK_STREAM
        self.type = type
        self.sock = None
 
    
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [2]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# conn = LazyConnection(('www.python.org', 443))
# Connection closed

with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    resp = resp.decode('utf-8')

print('RESP:', resp)

RESP: HTTP/1.1 301 Moved Permanently
Connection: close
Content-Length: 0
Server: Varnish
Retry-After: 0
Location: https://www.python.org/index.html
Accept-Ranges: bytes
Date: Thu, 11 Apr 2024 05:13:26 GMT
Via: 1.1 varnish
X-Served-By: cache-icn1450033-ICN
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1712812407.927358,VS0,VE0
Strict-Transport-Security: max-age=63072000; includeSubDomains; preload




In [12]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.connections = []
        
    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock

    
    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()
        

In [15]:
# Example use
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# conn = LazyConnection(('www.python.org', 443))
with conn as s1:
    pass
    with conn as s2:
        pass
    
# s1 and s2 are independent sockets

In [5]:
print('Hello World!')

Hello World!
